<a href="https://colab.research.google.com/github/PedroGalhardoB/ManipulacaoBancoDados/blob/main/Desafio15.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
using Distributed

# Adiciona processos workers
addprocs(4)

@everywhere function process_pair(i, j)
    # Simula algum processamento pesado
    result = i * j
    sleep(0.1)  # Simula trabalho
    return (i, j, result)
end

# Versão serial (similar ao seu exemplo)
function serial_product_example()
    println("=== VERSÃO SERIAL ===")
    @time for (i, j) in Iterators.product(1:5, 10:10:30)
        result = process_pair(i, j)
        println("i: $i, j: $j → resultado: $(result[3])")
    end
end

# Versão paralela
function parallel_product_example()
    println("\n=== VERSÃO PARALELA ===")
    @time results = @distributed vcat for (i, j) in collect(Iterators.product(1:5, 10:10:30))
        process_pair(i, j)
    end

    # Ordena os resultados para mesma ordem do serial
    sort!(results, by=x->(x[2], x[1]))  # ordena por j, depois i
    for (i, j, res) in results
        println("i: $i, j: $j → resultado: $res")
    end
end

# Executar
serial_product_example()
parallel_product_example()

=== VERSÃO SERIAL ===
i: 1, j: 10 → resultado: 10
i: 2, j: 10 → resultado: 20
i: 3, j: 10 → resultado: 30
i: 4, j: 10 → resultado: 40
i: 5, j: 10 → resultado: 50
i: 1, j: 20 → resultado: 20
i: 2, j: 20 → resultado: 40
i: 3, j: 20 → resultado: 60
i: 4, j: 20 → resultado: 80
i: 5, j: 20 → resultado: 100
i: 1, j: 30 → resultado: 30
i: 2, j: 30 → resultado: 60
i: 3, j: 30 → resultado: 90
i: 4, j: 30 → resultado: 120
i: 5, j: 30 → resultado: 150
  1.520599 seconds (1.58 k allocations: 220.117 KiB)

=== VERSÃO PARALELA ===
  7.166296 seconds (912.71 k allocations: 45.445 MiB, 0.19% gc time, 1 lock conflict, 19.19% compilation time)
i: 1, j: 10 → resultado: 10
i: 2, j: 10 → resultado: 20
i: 3, j: 10 → resultado: 30
i: 4, j: 10 → resultado: 40
i: 5, j: 10 → resultado: 50
i: 1, j: 20 → resultado: 20
i: 2, j: 20 → resultado: 40
i: 3, j: 20 → resultado: 60
i: 4, j: 20 → resultado: 80
i: 5, j: 20 → resultado: 100
i: 1, j: 30 → resultado: 30
i: 2, j: 30 → resultado: 60
i: 3, j: 30 → resultado: 90
i

In [6]:
using Base.Threads

@everywhere function process_pair(i, j)
    result = i * j
    sleep(0.1)
    return (i, j, result)
end

function serial_example()
    println("=== SERIAL ===")
    @time for (i, j) in Iterators.product(1:5, 10:10:30)
        process_pair(i, j)
    end
end

function parallel_example()
    println("\n=== PARALELO COM THREADS ===")
    @time @threads for (i, j) in collect(Iterators.product(1:5, 10:10:30))
        process_pair(i, j)
    end
end

println("Threads disponíveis: ", nthreads())
serial_example()
parallel_example()



println("Ao usar Distributed, o custo de comunicação entre processos superou o tempo da tarefa,")
println("tornando a versão paralela mais lenta.\n")
println("Já com threads, não há custo de comunicação, então o paralelismo foi eficiente.")


Threads disponíveis: 2
=== SERIAL ===
  1.518041 seconds (171 allocations: 8.633 KiB)

=== PARALELO COM THREADS ===
  0.841480 seconds (13.54 k allocations: 682.219 KiB, 7.24% compilation time)
Ao usar Distributed, o custo de comunicação entre processos superou o tempo da tarefa,
tornando a versão paralela mais lenta.

Já com threads, não há custo de comunicação, então o paralelismo foi eficiente.
